# Unnest and cleanup capacity field mapping
The human-entered mapping from Maastro clinic contains a lot of messy and nested data. We want to clean it, and unnest the data. WARNING: This is messy stuff, it seems to work in 95% of the cases, but there is a small group of diverse edgecases that still go wrong. We can NEVER guarantee that this data is converted correctly because there are sometimes just strings that need to be human-interpreted.

In [1]:
import pandas as pd
import numpy as np

In [2]:
cd ..

/Users/svenvanderburg/projects/capacity/ZIB-uploader


## Load excel file with mappings entered by humans from Maastro clinic
File can be found [on the Maastro teams](https://teams.microsoft.com/l/file/CADBAFA2-9EBA-4E19-B8E4-935EA2A7974B?tenantId=b9c5dd0d-b3a0-4572-a670-e4e8bd6c9350&fileType=xlsx&objectUrl=https%3A%2F%2Fmaastro.sharepoint.com%2Fsites%2FO365GroepCDSCARRIERProjectManagement%2FShared%20Documents%2FCAPACITY%2FCodes%20mapping%20(1).xlsx&baseUrl=https%3A%2F%2Fmaastro.sharepoint.com%2Fsites%2FO365GroepCDSCARRIERProjectManagement&serviceName=teams&threadId=19:373f883c850c4e9aa697817c3cc38676@thread.skype&groupId=257a7bbf-43a5-4906-ab8e-c4deb714fc6d) or [on their sharepoint](https://maastro.sharepoint.com/sites/O365GroepCDSCARRIERProjectManagement/_layouts/15/Doc.aspx?OR=teams&action=edit&sourcedoc={CADBAFA2-9EBA-4E19-B8E4-935EA2A7974B}). Talk to Djura for access.

In [81]:
df = pd.read_excel('data/capacity-human-entered-mapping.xlsx', sheet_name='CAPACITY - required variables')

In [82]:
df.head(4)

,Variable/Field Name,Field Label,Fiel Attributes,Zib,Zib concept,Zib codesystem,EHR,Snomed,COD016-VEKT code,ICD-10,ATC
0,participant_identification_number_required,Participant Identification Number Pin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,subjid,Participant Identification Number (PIN),text,Patient,PatientIdentificationNumber,NaN,pseudonym,NaN,NaN,NaN,NaN
2,studyid,STUDY ID,text,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,participant_identification_number_pin_required...,Comple?,"0-Incomplete,1-Unverified,2-Complete",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Cleanup data

In [83]:
# Rename columns 
df.columns = ['capacity_field_name', 'capacity_field_description', 'capacity_field_attributes', 'zib', 'zib_concept', 'zib_codesystem', 'ehr', 'snomed', 'cod016_vekt_code', 'icd_10_code', 'atc']

In [84]:
df.head(6)

,capacity_field_name,capacity_field_description,capacity_field_attributes,zib,zib_concept,zib_codesystem,ehr,snomed,cod016_vekt_code,icd_10_code,atc
0,participant_identification_number_required,Participant Identification Number Pin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,subjid,Participant Identification Number (PIN),text,Patient,PatientIdentificationNumber,NaN,pseudonym,NaN,NaN,NaN,NaN
2,studyid,STUDY ID,text,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,participant_identification_number_pin_required...,Comple?,"0-Incomplete,1-Unverified,2-Complete",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,inclusion_criteria_required,Inclusion Criteria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,dsstdat,Date of Enrollment,text(date),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Explode `capacity_field_attributes` column

In [85]:
df['capacity_field_attributes'] = df['capacity_field_attributes'].str.split(r';|,')

In [86]:
df = df.explode('capacity_field_attributes')
df = df.reset_index(drop=True)

In [87]:
df.head(10)

,capacity_field_name,capacity_field_description,capacity_field_attributes,zib,zib_concept,zib_codesystem,ehr,snomed,cod016_vekt_code,icd_10_code,atc
0,participant_identification_number_required,Participant Identification Number Pin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,subjid,Participant Identification Number (PIN),text,Patient,PatientIdentificationNumber,NaN,pseudonym,NaN,NaN,NaN,NaN
2,studyid,STUDY ID,text,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,participant_identification_number_pin_required...,Comple?,0-Incomplete,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,participant_identification_number_pin_required...,Comple?,1-Unverified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,participant_identification_number_pin_required...,Comple?,2-Complete,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,inclusion_criteria_required,Inclusion Criteria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,dsstdat,Date of Enrollment,text(date),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,sitename_nhr,Site name,NaN,HealthcareProvider,OrganizationName,NaN,NaN,NaN,NaN,NaN,NaN
9,country,Country,text (?),AddressInformation,Country,CountryGBACodelist or CountryISOCodelist,NaN,NaN,NaN,NaN,NaN


### Split `capacity_field_attributes` column into `capacity_field_choice_code` and `capacity_field_choice_description`

In [88]:
def _extract_choice(string, index):
    if pd.isna(string):
        return string
    if '-1' in string:
        # Sometimes -1 is in the string, we can therefore not
        # split on '-', but need to manually do this:
        splitted = ['-1', string.strip('-1 ')]
    else:
        splitted = string.split('-')
        if len(splitted) == 1:
            return np.nan
    return splitted[index].strip()


def extract_choice_code(string):
    """
    i.e. extracts 3 from '3-something'
    """
    return _extract_choice(string, 0)


def extract_choice_description(string):
    """
    i.e. extracts something from '3-something'
    """
    return _extract_choice(string, 1)
    
df['capacity_field_choice_code'] = df['capacity_field_attributes'].apply(extract_choice_code)
df['capacity_field_choice_description'] = df['capacity_field_attributes'].apply(extract_choice_description)

In [89]:
df[['capacity_field_choice_code', 'capacity_field_choice_description', 'capacity_field_attributes']].head(20)

,capacity_field_choice_code,capacity_field_choice_description,capacity_field_attributes
0,NaN,NaN,NaN
1,NaN,NaN,text
2,NaN,NaN,text
3,0,Incomplete,0-Incomplete
4,1,Unverified,1-Unverified
5,2,Complete,2-Complete
6,NaN,NaN,NaN
7,NaN,NaN,text(date)
8,NaN,NaN,NaN
9,NaN,NaN,text (?)


In [90]:
df = df.drop(columns=['capacity_field_attributes'])

### Split `snomed` and `atc` column

In [91]:
df[~pd.isna(df['snomed'])][['capacity_field_name', 'snomed', 'capacity_field_choice_code', 'capacity_field_choice_description']]

,capacity_field_name,snomed,capacity_field_choice_code,capacity_field_choice_description
44,healthwork_erterm,223366009 | Healthcare professional (occupation),1,yes
45,healthwork_erterm,223366009 | Healthcare professional (occupation),2,no
46,healthwork_erterm,223366009 | Healthcare professional (occupation),3,N/A
47,labwork_erterm,159138004 | Microbiologist (occupation),1,yes
48,labwork_erterm,159138004 | Microbiologist (occupation),2,no
...,...,...,...,...
430,carhist_valve,60573004 | Aortic valve stenosis (disorder) |6...,3,Mitral valve stenosis
431,carhist_valve,60573004 | Aortic valve stenosis (disorder) |6...,4,Mitral valve regurgitation
432,carhist_valve,60573004 | Aortic valve stenosis (disorder) |6...,5,Pulmonary valve regurgitation
433,carhist_valve,60573004 | Aortic valve stenosis (disorder) |6...,7,Tricuspid valve stenosis


The snomed and atc fields correspond of pipe-separated text depicting the code and description:

In [92]:
df.iloc[430]['snomed']

'60573004 | Aortic valve stenosis (disorder) |60234000 | Aortic valve regurgitation (disorder) |48724000 | Mitral valve regurgitation (disorder) |91434003 | Pulmonic valve regurgitation (disorder) 49915006 | Tricuspid valve stenosis (disorder) |111287006 | Tricuspid valve regurgitation (disorder) |'

In [93]:
def _extract_pipe_separated(group, column):
    pipe_separated_text = group[column].iloc[0]
    if pd.isna(pipe_separated_text):
        codes = [np.nan] * len(group)
        descriptions = [np.nan] * len(group)
    else:
        splitted = pipe_separated_text.strip('|').split('|')
        
        if len(splitted) == 2:
            # In this case the pipe separated text just applies to all rows in the group
            codes = [splitted[0].strip()] * len(group)
            descriptions = [splitted[1].strip()] * len(group)
        else:
            # The pipe separated text is referring to different rows in the group
            # i.e. 386661006 | Fever (finding)|49727002 | Cough (finding) correspond to
            # 2 different rows
            codes = list()
            descriptions = list()
            first_element_none = False
            for i, (_, row) in enumerate(group.iterrows()):
                if i == 0 and row['capacity_field_choice_description'].lower() == 'none':
                    first_element_none = True
                try:
                    # If the first element is none there is no entry for it in the pipe separated field,
                    # so the indexes mismatch by 1 element
                    code = splitted[(i - first_element_none)* 2] 
                    description = splitted[(i - first_element_none) * 2 + 1]
                except IndexError:
                    code = np.nan
                    description = np.nan
                codes.append(code)
                descriptions.append(description)
    group[f'{column}_code'] = codes
    group[f'{column}_description'] = descriptions
    return group

def extract_snomed(group):
    _extract_pipe_separated(group, 'snomed')
    return group

def extract_atc(group):
    _extract_pipe_separated(group, 'atc')
    return group

df = df.groupby('capacity_field_name').apply(extract_snomed)
df = df.groupby('capacity_field_name').apply(extract_atc)

In [96]:
df[~pd.isna(df['snomed'])][['capacity_field_name', 'capacity_field_choice_description', 'snomed', 'snomed_code', 'snomed_description']].head(10)

,capacity_field_name,capacity_field_choice_description,snomed,snomed_code,snomed_description
44,healthwork_erterm,yes,223366009 | Healthcare professional (occupation),223366009,Healthcare professional (occupation)
45,healthwork_erterm,no,223366009 | Healthcare professional (occupation),223366009,Healthcare professional (occupation)
46,healthwork_erterm,N/A,223366009 | Healthcare professional (occupation),223366009,Healthcare professional (occupation)
47,labwork_erterm,yes,159138004 | Microbiologist (occupation),159138004,Microbiologist (occupation)
48,labwork_erterm,no,159138004 | Microbiologist (occupation),159138004,Microbiologist (occupation)
49,labwork_erterm,N/A,159138004 | Microbiologist (occupation),159138004,Microbiologist (occupation)
99,admis_complaints,Fever (t>38C or >100.4F),386661006 | Fever (finding)|49727002 | Cough (...,386661006,Fever (finding)
100,admis_complaints,Cough,386661006 | Fever (finding)|49727002 | Cough (...,49727002,Cough (finding)
101,admis_complaints,Sore throat,386661006 | Fever (finding)|49727002 | Cough (...,162397003,Pain in throat (finding)
102,admis_complaints,Dyspnea,386661006 | Fever (finding)|49727002 | Cough (...,67036007,Dyspnea (finding)


In [98]:
df[~pd.isna(df['atc'])][['capacity_field_name', 'capacity_field_choice_description', 'atc', 'atc_code', 'atc_description']].head(10)

,capacity_field_name,capacity_field_choice_description,atc,atc_code,atc_description
202,med_nsaid,None,M01AE01|ibuprofen|M01AE02|naproxen|M01AB05|Dic...,M01AB05,Diclofenac
203,med_nsaid,Ibuprofen,M01AE01|ibuprofen|M01AE02|naproxen|M01AB05|Dic...,M01AE01,ibuprofen
204,med_nsaid,Naproxen,M01AE01|ibuprofen|M01AE02|naproxen|M01AB05|Dic...,M01AE02,naproxen
205,med_nsaid,Diclofenac,M01AE01|ibuprofen|M01AE02|naproxen|M01AB05|Dic...,M01AB05,Diclofenac
206,med_nsaid,1,M01AE01|ibuprofen|M01AE02|naproxen|M01AB05|Dic...,NaN,NaN
216,carmed,None,C07A|Beta blocking agents|C01B|ANTIARRHYTHMICS...,A10B,"bloog glucose lowering drugs, excl. insulins(O..."
217,carmed,Betablocker,C07A|Beta blocking agents|C01B|ANTIARRHYTHMICS...,C07A,Beta blocking agents
218,carmed,Antiarrhytmic drugs,C07A|Beta blocking agents|C01B|ANTIARRHYTHMICS...,C01B,"ANTIARRHYTHMICS, CLASS I AND III"
219,carmed,Digoxine,C07A|Beta blocking agents|C01B|ANTIARRHYTHMICS...,C01AA05,digoxin
220,carmed,Diuretics,C07A|Beta blocking agents|C01B|ANTIARRHYTHMICS...,C03,Diuretics


In [107]:
df.columns

Index(['capacity_field_name', 'capacity_field_description', 'zib',
       'zib_concept', 'zib_codesystem', 'ehr', 'cod016_vekt_code',
       'icd_10_code', 'capacity_field_choice_code',
       'capacity_field_choice_description', 'snomed_code',
       'snomed_description', 'atc_code', 'atc_description'],
      dtype='object')

In [108]:
# Final set of columns
df = df[['capacity_field_name', 'capacity_field_description', 'capacity_field_choice_code',
         'capacity_field_choice_description', 'zib', 'zib_concept', 'zib_codesystem', 
         'snomed_code', 'snomed_description', 'atc_code', 'atc_description', 'ehr',
         'cod016_vekt_code', 'icd_10_code']]

In [109]:
df.to_csv('data/capacity-human-entered-mapping-cleaned.csv', index=False)